## Generate a mapping file with UUIDs for use in openLCA
1. Run cell 1 to generate the upload button. Select your mapping file by clicking upload. The mapping file must be csv file that conforms to the mapping standard.
2. Run cell 2 to add UUIDs. Conversion factors will be applied for any mappings that change units for which the alternate unit exists within the FEDEFL. The outcome will be printed on screen.
3. Run cell 3 to generate a link for downloading the json mapping file (.zip) for use in openLCA

_To restart, select Kernel --> "Restart and Clear Output"_

In [ ]:
## Step 1
import pandas as pd
import fedelemflowlist
from fedelemflowlist.globals import flowmappingpath, flowmapping_fields
import os
import ipywidgets as widgets
from IPython.display import display, FileLink

if not os.path.exists('./scratch'):
    os.mkdir('./scratch')

uploader = widgets.FileUpload(
     accept   = '.csv'
    ,multiple = False
)

display(uploader)

### Select the upload button above to upload your csv mapping file
Then proceed to the next cell

In [ ]:
## Step 2
if not uploader.value:
    raise ValueError('Input Excel file is required to continue. Make sure to select the Upload button above before continuing.')

input_file = next(iter(uploader.value))
input_full = r"./scratch/" + input_file

if os.path.exists(input_full):
    os.remove(input_full)
    
with open(input_full,"wb") as fp:
    fp.write(uploader.value[input_file]['content'])

mapping = pd.read_csv(input_full)
mapping_length = len(mapping)
print(str(mapping_length) + ' flows in the mapping file.')
all_flows = fedelemflowlist.get_flows()
all_flows = all_flows[['Flowable', 'Context', 'Flow UUID', 'Unit']]
mapping_w_flowinfo = pd.merge(mapping, all_flows,
            left_on=['TargetFlowName', 'TargetFlowContext', 'TargetUnit'],
            right_on=['Flowable', 'Context', 'Unit'])
mapping_w_flowinfo = mapping_w_flowinfo.drop(columns=['TargetFlowUUID', 'Flowable',
                                        'Context', 'Unit'])
mapping_w_flowinfo = mapping_w_flowinfo.rename(columns={'Flow UUID': 'TargetFlowUUID'})
mapping_merged_len = len(mapping_w_flowinfo)
if mapping_length > mapping_merged_len:
    print("Not all flows were mapped to flows in the list")
print('UUIDs added for ' + str(mapping_merged_len) + ' flows.')

conversions = fedelemflowlist.get_alt_conversion()
# merge in conversion factors where source unit = alternate unit
mapping_w_conversion = pd.merge(mapping_w_flowinfo, conversions, how='left',
                              left_on=['TargetFlowName', 'SourceUnit', 'TargetUnit'],
                              right_on=['Flowable', 'AltUnit', 'Unit'])
# update conversion factor where current conversion is 1 and the updated conversion exists
converted1 = mapping_w_conversion['InverseConversionFactor'].notnull() 
converted2 = mapping_w_conversion['ConversionFactor']==1
mapping_w_conversion['Convert']=converted1 & converted2
mapping_w_conversion.loc[(mapping_w_conversion['Convert']==True), 
                         'ConversionFactor']=mapping_w_conversion['InverseConversionFactor']
converted = mapping_w_conversion['Convert'].sum()
print('Added conversion factors for ' + str(converted) + ' flows.')
mapping_w_conversion = mapping_w_conversion.drop(columns=['Flowable','Unit',
                                                     'AltUnit','AltUnitConversionFactor',
                                                     'InverseConversionFactor', 'Convert'])
flowmapping_order = list(flowmapping_fields.keys())
mapping_w_conversion =  mapping_w_conversion[flowmapping_order]
print(mapping_w_conversion)
    

In [ ]:
## Step 3
output_file = 'flows_w_mappings.zip'
output_full = r"scratch/" + output_file

if os.path.exists(output_full):
   os.remove(output_full)

mapping_flow_uuids = pd.DataFrame(pd.unique(mapping_w_conversion['TargetFlowUUID']),columns=["Flow UUID"])

#Get all flows
all_flows = fedelemflowlist.get_flows()
#Subset all flows to get just those used in selected mapping
flows_used_in_mapping =  pd.merge(all_flows,mapping_flow_uuids)

fedelemflowlist.write_jsonld(flows_used_in_mapping,output_full,mapping_w_conversion)
print("mapping file created as json")
download = FileLink(output_full,result_html_prefix="Click here to download: ")
display(download)
